# Import Libraries

In [4]:
import torch
import torch.nn.functional as F
from facenet_pytorch import MTCNN, InceptionResnetV1
import numpy as np
from PIL import Image
import cv2
import warnings
import gradio as gr

from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

warnings.filterwarnings("ignore")


Download and Load Model

In [5]:
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

mtcnn = MTCNN(
    select_largest=False,
    post_process=False,
    device=DEVICE
).eval().to(DEVICE)


In [6]:
model = InceptionResnetV1(
    pretrained="vggface2",
    classify=True,
    num_classes=1,
    device=DEVICE
)

# Load checkpoint
checkpoint = torch.load("resnetinceptionv1_epoch_32.pth", map_location='cpu')
model.load_state_dict(checkpoint['model_state_dict'])
model.eval().to(DEVICE)


FileNotFoundError: [Errno 2] No such file or directory: 'resnetinceptionv1_epoch_32.pth'

# Model Inference 

In [7]:
def predict(input_image: Image.Image, threshold: float = 0.5):
    # Detect face
    face = mtcnn(input_image)
    if face is None:
        raise Exception('No face detected')

    # Resize
    face = face.unsqueeze(0)
    face = F.interpolate(face, size=(256, 256), mode='bilinear', align_corners=False)

    # Save original for visualization
    prev_face = face.squeeze(0).permute(1, 2, 0).cpu().detach().int().numpy().astype('uint8')

    # Normalize
    face = face.to(DEVICE).float() / 255.0
    face_image_to_plot = face.squeeze(0).permute(1, 2, 0).cpu().detach().numpy()

    # Grad-CAM
    target_layers = [model.block8.branch1[-1]]
    cam = GradCAM(model=model, target_layers=target_layers, use_cuda=torch.cuda.is_available())
    targets = [ClassifierOutputTarget(0)]
    grayscale_cam = cam(input_tensor=face, targets=targets, eigen_smooth=True)[0]
    visualization = show_cam_on_image(face_image_to_plot, grayscale_cam, use_rgb=True)
    face_with_mask = cv2.addWeighted(prev_face, 1, visualization, 0.5, 0)

    # Predict
    with torch.no_grad():
        output = torch.sigmoid(model(face).squeeze(0))
        prediction = "real" if output.item() < threshold else "fake"

        confidences = {
            'real': 1 - output.item(),
            'fake': output.item()
        }

    return confidences, Image.fromarray(face_with_mask)


# Gradio Interface

In [ ]:
interface = gr.Interface(
    fn=predict,
    inputs=[
        gr.inputs.Image(label="Input Image", type="pil"),
        gr.inputs.Slider(minimum=0.0, maximum=1.0, default=0.5, label="Prediction Threshold")
    ],
    outputs=[
        gr.outputs.Label(label="Prediction Confidence"),
        gr.outputs.Image(label="Face with Explainability", type="pil")
    ],
    title="Deepfake Detection with Explainability",
    description="Upload an image. Adjust the threshold for real/fake classification. CAM shows model focus."
)

interface.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 3.39.0, however version 4.44.1 is available, please upgrade.
--------
